In [8]:
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


class Knude:
    '''
    Defines a class for a tree node (Danish: Knude)
    '''
    def __init__(self, feature=None, threshold=None, inf_gain=None, classification=None, left=None, right=None):
        self.feature = feature #which feature are we splitting on?
        self.threshold = threshold #which value of the feature?
        self.inf_gain = inf_gain #infogain
        self.classification = classification #classification of leaf node
        self.left = left #left partition
        self.right = right #right partition

class DecisionTree:
    '''
    Implements the DT as a class. Default vals for min_leaf_size
    and max_height chosen approximately as
    min_leaf_size=n^(1/3)
    max_height=features^(1/2)
    '''
    def __init__(self, min_leaf_size=36, max_height=4,randomized_features=False):
        self.root = None
        self.randomized_features=randomized_features
        self.min_leaf_size = min_leaf_size
        self.max_height = max_height

    @staticmethod
    def _compute_entropy(p):
        '''
        Helper function to compute entropy based on list of p integer values

        Arguments:
        p: list

        Returns:
        entropy: float
        '''
        count = np.bincount(np.array(p, dtype=np.int64)) # runtime error w/o np.int64
        probs = count / len(p)  # prob of each label
        entropy = -np.sum(probs * np.log2(probs, where=probs > 0))
        return entropy

    def _compute_information_gain(self, parent, left_branch, right_branch):
        '''
        Helper function, computes information_gain

        Arguments:
        parent: list
        left_branch: list
        right_branch: list

        Returns:
        information_gain: float

        '''
        left_frac = len(left_branch) / len(parent)
        right_frac = len(right_branch) / len(parent)

        information_gain = self._compute_entropy(parent) - (left_frac * self._compute_entropy(left_branch) + right_frac * self._compute_entropy(right_branch))
        return information_gain

    def _compute_best_split(self, X, y):
        '''
        Helper function, takes X and y as inputs and finds the feature and values to split on to 
        get the biggest information gain. If randomized_features==True, it will only search
        through a randomized subset of the columns, otherwise it will search through 
        all columns

        Arguments:
        X: np.array, data
        y: np.array, targets

        Returns:
        best_split: dict, info for the node

        '''
        
        n_cols = X.shape[1]
        best_split = {}
        best_info_gain = -2
        
        if self.randomized_features==True:
            tweak_size=int(np.sqrt(n_cols))
            chosen_cols=np.random.choice(a=range(n_cols),size=tweak_size,replace=True)
        elif self.randomized_features==False:
            chosen_cols=range(n_cols)
            

        for feature in chosen_cols:
            X_curr = X[:, feature] #choose splitting feature
            split_vals=np.unique(X_curr)
            for split_val in split_vals: #check every unique val in feature

                #split X into higher and lower than given unique val "threshold"
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                left = np.array([r for r in df if r[feature] <= split_val])
                right = np.array([r for r in df if r[feature] > split_val])

                
                if len(left) > 0 and len(right) > 0: # If all data is in one df, no need to test
                    # extract target vals as last column
                    y = df[:, -1]
                    y_left = left[:, -1]
                    y_right = right[:, -1]
                    
                    inf_gain = self._compute_information_gain(y, y_left, y_right)

                    if inf_gain > best_info_gain: #save if new split is better than previous best
                        best_split = { # save info for node assignment
                            'feature_index': feature,
                            'threshold': split_val,
                            'df_left': left,
                            'df_right': right,
                            'inf_gain': inf_gain
                        }
                        best_info_gain = inf_gain
        
        return best_split

    def _build_tree(self, X, y, height=0):
        '''
        Helper function, builds DT for one point

        Arguments:
        X: np.array, data
        y: np.array, targets
        height: int, for stopping criteria

        Returns:
        node: class instance
        '''

        n_rows = X.shape[0]
        # Get the best split
        best = self._compute_best_split(X, y)
        
        # Check to see if a node should be leaf node
        if n_rows >= self.min_leaf_size and height <= self.max_height and bool(best):
            if best['inf_gain'] > 0:
                # Build a tree on the left
                left = self._build_tree(
                    X=best['df_left'][:, :-1],
                    y=best['df_left'][:, -1],
                    height=height + 1
                )
                right = self._build_tree(
                    X=best['df_right'][:, :-1],
                    y=best['df_right'][:, -1],
                    height=height + 1
                )
                return Knude(
                    feature=best['feature_index'],
                    threshold=best['threshold'],
                    left=left,
                    right=right,
                    inf_gain=best['inf_gain']
                )
        # Leaf node
        return Knude(
            classification=Counter(y).most_common(1)[0][0]
        )

    def fit(self, X, y):
        '''
        Builds DT

        Arguments:
        X: np.array, features
        y: np.array, targets
        '''
        # Call _build_tree starting from the root
        self.root = self._build_tree(X, y)

    def _predict_instance(self, x, tree):
        '''
        Helper function, predicts classification recursevely for one point

        Arguments:
        x: np.array, single observation
        tree: the trained tree

        Returns:
        Class prediction: float

        '''
        # Checks if its a node
        if tree.classification != None:
            return tree.classification

        X_feature = x[tree.feature] #extracts the feature to be split on

        if X_feature <= tree.threshold:
            return self._predict_instance(x=x, tree=tree.left) #get prediction from left part

        if X_feature > tree.threshold:
            return self._predict_instance(x=x, tree=tree.right) #get prediction from right part

    def predict(self, X):
        '''
        Function calling predict for all instances

        Arguments:
        X: np.array, data

        Returns:
        Class prediction: np.array
        '''
        return [self._predict_instance(x, self.root) for x in X] #calls _predict_instance for every instance

class RandomForest:
    def __init__(self, randomized_features=False,n_trees=10, max_height=5,min_leaf_size=5):
        self.randomized_features=randomized_features
        self.n_trees = n_trees
        self.max_height = max_height
        self.min_leaf_size = min_leaf_size
        # Will store individually trained decision trees
        self.tree_list = []

    @staticmethod
    def _bootstrap(X, y):
        '''
        Bootstraps based on X and y
        '''
        # Sample with replacement
        samples = np.random.choice(a=X.shape[0], size=X.shape[0], replace=True)
        feature_sample=X[samples]
        target_sample=y[samples]
        return feature_sample,target_sample

    def fit(self, X, y):
        '''
        Trains the RF classifier by building n_trees decision trees

        Arguments:
        X: np.array, features
        y: np.array, target
        '''
        # Reset
        if len(self.tree_list) > 0:
            self.tree_list = []

        # Build each tree of the forest
        trees_built = 0
        while trees_built < self.n_trees:
            try:
                print("beginning to build tree nr", trees_built)
                dt = DecisionTree(
                    min_leaf_size = self.min_leaf_size,
                    max_height = self.max_height,randomized_features = self.randomized_features
                )
                # Obtain data sample
                X_bootstrapped, y_bootstrapped = self._bootstrap(X, y)
                # Train
                dt.fit(X_bootstrapped, y_bootstrapped)
                # Save the classifier
                self.tree_list.append(dt)
                print("built tree nr",trees_built)
                trees_built = trees_built+1
            except Exception as e:
                print(e)
                continue

    def predict(self, X):
        '''
        Predicts labels for all test instances

        Arguments:
        X: np.array, instances to predict

        Returns:
        Predictions: np.array, predictions for each instance
        '''
        # Make predictions with every tree in the forest
        predictions = [tree.predict(X) for tree in self.tree_list]
        # Use majority voting for the final prediction
        prediction = [max(Counter(preds), key=Counter(preds).get) for preds in np.swapaxes(a=predictions, axis1=0, axis2=1)]
        return prediction


In [3]:
######## DATA LOADING AND CLEANING #############
data = pd.read_csv('Hotel Reservations.csv') #load data
data_encode = data.copy()
#One hot encode
labels_to_encode = ['type_of_meal_plan', 'room_type_reserved',
                    'market_segment_type']
data_encode = pd.get_dummies(data, columns = labels_to_encode) 

data_encode['booking_status'] = data_encode['booking_status'].astype('category')
data_encode['booking_status'] = data_encode['booking_status'].cat.codes
data_encode = data_encode[data_encode['no_of_weekend_nights']+data_encode["no_of_week_nights"] > 0]

X = np.array(data_encode.drop(['Booking_ID', 'booking_status'], axis=1))
y = np.array(data_encode['booking_status'])

#sample_size=5000

#X=X[:sample_size,:]
#y=y[:sample_size]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)


data_feature_engineered=data_encode.copy()
data_feature_engineered["total_nights"]=data_encode["no_of_weekend_nights"]+data_encode["no_of_week_nights"]
data_feature_engineered["total_guests"]=data_encode["no_of_adults"]+data_encode["no_of_children"]
data_feature_engineered["total_bookings"]=data_encode["no_of_previous_cancellations"]+data_encode["no_of_previous_bookings_not_canceled"]
data_feature_engineered["total_cost"]=data_feature_engineered["total_nights"]*data_feature_engineered["avg_price_per_room"]

X_feature_engineered = np.array(data_feature_engineered.drop(['Booking_ID', 'booking_status'], axis=1))
y_feature_engineered = np.array(data_feature_engineered['booking_status'])

#X_feature_engineered=X_feature_engineered[:sample_size,:]
#y_feature_engineered=y_feature_engineered[:sample_size]

X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(X_feature_engineered, y_feature_engineered, test_size=0.2, random_state=2)

In [ ]:
############ PCA ############
from sklearn.preprocessing import scale 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)
X_train_s=scaler.transform(X_train)
X_test_s=scaler.transform(X_test)

pca=PCA(0.95)
pca.fit(X_train_s)
X_pca_train = pca.transform(X_train_s)
X_pca_test = pca.transform(X_test_s)

In [ ]:
############ TRAINING MODEL ON OUR IMPLEMENTATION ######
model_dt = DecisionTree()
model_dt.fit(X_train, y_train)

preds_train = model_dt.predict(X_train)
preds_test = model_dt.predict(X_test)

print("DT train accuracy:",accuracy_score(y_train, preds_train))
print("DT test accuracy:",accuracy_score(y_test, preds_test))

In [ ]:
############ BASE RANDOM FOREST ######
start = timeit.default_timer()
print("Training base random forest")
model_rf = RandomForest(n_trees=16,min_leaf_size=4,max_height=32)
model_rf.fit(X_train, y_train)

preds_train = model_rf.predict(X_train)
preds_test = model_rf.predict(X_test)
print("Base random forrest train accuracy:",accuracy_score(y_train, preds_train))
print("Base random forrest test accuracy:",accuracy_score(y_test, preds_test))
stop = timeit.default_timer()
print('Time: ', stop - start) 

In [ ]:
############ BASE RANDOM FOREST WITH PCA######
print("Training base random forest")
start = timeit.default_timer()
model_rf = RandomForest(n_trees=16,min_leaf_size=4,max_height=32)
model_rf.fit(X_pca_train, y_train)

preds_train = model_rf.predict(X_pca_train)
preds_test = model_rf.predict(X_pca_test)

stop = timeit.default_timer()
print("Base random forrest train accuracy:",accuracy_score(y_train, preds_train))
print("Base random forrest test accuracy:",accuracy_score(y_test, preds_test))
print('Time: ', stop - start) 

In [4]:
############ FEATURE ENGINEREERED RANDOM FOREST ######
start = timeit.default_timer()
print("Training base random forrest for feature engineered data set")
model_fe = RandomForest(n_trees=16,min_leaf_size=4,max_height=32)
model_fe.fit(X_train_fe, y_train_fe)

preds_train_fe = model_fe.predict(X_train_fe)
preds_test_fe = model_fe.predict(X_test_fe)

stop = timeit.default_timer()

print("FE data train accuracy:",accuracy_score(y_train_fe, preds_train_fe))
print("FE data test accuracy:",accuracy_score(y_test_fe, preds_test_fe))
print('Time: ', stop - start) 

Training base random forrest for feature engineered data set
beginning to build tree nr 0
built tree nr 0
beginning to build tree nr 1
built tree nr 1
beginning to build tree nr 2
built tree nr 2
beginning to build tree nr 3
built tree nr 3
beginning to build tree nr 4
built tree nr 4
beginning to build tree nr 5
built tree nr 5
beginning to build tree nr 6
built tree nr 6
beginning to build tree nr 7
built tree nr 7
beginning to build tree nr 8
built tree nr 8
beginning to build tree nr 9
built tree nr 9
beginning to build tree nr 10
built tree nr 10
beginning to build tree nr 11
built tree nr 11
beginning to build tree nr 12
built tree nr 12
beginning to build tree nr 13
built tree nr 13
beginning to build tree nr 14
built tree nr 14
beginning to build tree nr 15
built tree nr 15
FE data train accuracy: 0.9887764616500329
FE data test accuracy: 0.8940607734806629
Time:  10794.548590400002


In [9]:
############ FEATURE ENGINEREERED RANDOM FOREST WITH RANDOMIZATION ######
print("Training randomized forrest for feature engineered data set")
start = timeit.default_timer()
model_rand = RandomForest(n_trees=16,min_leaf_size=4,max_height=32,randomized_features=True)
model_rand.fit(X_train, y_train)

preds_train_rand = model_rand.predict(X_train)
preds_test_rand = model_rand.predict(X_test)

stop = timeit.default_timer()
print("Random data train accuracy:",accuracy_score(y_train, preds_train_rand))
print("Random data test accuracy:",accuracy_score(y_test, preds_test_rand))
print('Time: ', stop - start) 

Training randomized forrest for feature engineered data set
beginning to build tree nr 0


In [5]:
############ FEATURE ENGINEREERED RANDOM FOREST WITH RANDOMIZATION ######
print("Training randomized forrest for feature engineered data set")
start = timeit.default_timer()
model_fe_rand = RandomForest(n_trees=16,min_leaf_size=4,max_height=32,randomized_features=True)
model_fe_rand.fit(X_train_fe, y_train_fe)

preds_train_rand = model_fe_rand.predict(X_train_fe)
preds_test_rand = model_fe_rand.predict(X_test_fe)

stop = timeit.default_timer()
print("FE data train accuracy:",accuracy_score(y_train_fe, preds_train_rand))
print("FE data test accuracy:",accuracy_score(y_test_fe, preds_test_rand))
print('Time: ', stop - start) 

Training randomized forrest for feature engineered data set
beginning to build tree nr 0
built tree nr 0
beginning to build tree nr 1
built tree nr 1
beginning to build tree nr 2
built tree nr 2
beginning to build tree nr 3
built tree nr 3
beginning to build tree nr 4
built tree nr 4
beginning to build tree nr 5
built tree nr 5
beginning to build tree nr 6
built tree nr 6
beginning to build tree nr 7
built tree nr 7
beginning to build tree nr 8
built tree nr 8
beginning to build tree nr 9
built tree nr 9
beginning to build tree nr 10
built tree nr 10
beginning to build tree nr 11
built tree nr 11
beginning to build tree nr 12
built tree nr 12
beginning to build tree nr 13
built tree nr 13
beginning to build tree nr 14
built tree nr 14
beginning to build tree nr 15
built tree nr 15
FE data train accuracy: 0.9256483751769866
FE data test accuracy: 0.8863259668508288
Time:  2143.4467369000013


In [ ]:
#PCA random forest, timetest
X=X[0:500,]
y=y[0:500]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

#Do PCA
from sklearn.preprocessing import scale 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
X_train_s=scaler.transform(X_train)
X_test_s=scaler.transform(X_test)
pca=PCA(0.95)
pca.fit(X_train_s)
X_pca_train = pca.transform(X_train_s)
X_pca_test = pca.transform(X_test_s)


#Random forest with PCA
import timeit
start = timeit.default_timer()
max_features=None
rf = RandomForest(n_trees=16,max_height=32,min_leaf_size=4)
rf.fit(X_pca_train, y_train)
rf_train_accuracy = accuracy_score(y_train, rf.predict(X_pca_train))
rf_test_accuracy = accuracy_score(y_test, rf.predict(X_pca_test))
print("train", rf_train_accuracy, "test",rf_test_accuracy)
stop = timeit.default_timer()
print('Time with PCA: ', stop - start) 

#Random forest without
import timeit
start = timeit.default_timer()
max_features=None
rf = RandomForest(n_trees=16,max_height=32,min_leaf_size=4)
rf.fit(X_train, y_train)
rf_train_accuracy = accuracy_score(y_train, rf.predict(X_train))
rf_test_accuracy = accuracy_score(y_test, rf.predict(X_test))
print("train", rf_train_accuracy, "test",rf_test_accuracy)
stop = timeit.default_timer()
print('Time without PCA: ', stop - start)